In [1]:
import os
import logging
from os.path import abspath
from pathlib import Path
from dataclasses import field
from typing import Optional
from datasets import load_dataset
import transformers
import torch
from transformers import (
    T5Tokenizer,
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    HfArgumentParser,
    TrainingArguments,
)
from wasabi import msg

/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


## Load dataset

In [2]:
dataset = load_dataset(
        'csv', 
        data_dir="data/cdr_seq2rel",
        column_names=['input', 'relations']
        )

dataset_train = dataset['train'].select(range(1,501)) # remove first row that contains column names
dataset_eval = dataset['validation'].select(range(1,501)) # remove first row that contains column names

In [3]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
show_random_elements(dataset_train)

,input,relations
0,"Prenatal dexamethasone programs hypertension and renal injury in the rat. Dexamethasone is frequently administered to the developing fetus to accelerate pulmonary development. The purpose of the present study was to determine if prenatal dexamethasone programmed a progressive increase in blood pressure and renal injury in rats. Pregnant rats were given either vehicle or 2 daily intraperitoneal injections of dexamethasone (0.2 mg/kg body weight) on gestational days 11 and 12, 13 and 14, 15 and 16, 17 and 18, or 19 and 20. Offspring of rats administered dexamethasone on days 15 and 16 gestation had a 20% reduction in glomerular number compared with control at 6 to 9 months of age (22 527+/-509 versus 28 050+/-561, P<0.05), which was comparable to the percent reduction in glomeruli measured at 3 weeks of age. Six- to 9-month old rats receiving prenatal dexamethasone on days 17 and 18 of gestation had a 17% reduction in glomeruli (23 380+/-587) compared with control rats (P<0.05). Male rats that received prenatal dexamethasone on days 15 and 16, 17 and 18, and 13 and 14 of gestation had elevated blood pressures at 6 months of age; the latter group did not have a reduction in glomerular number. Adult rats given dexamethasone on days 15 and 16 of gestation had more glomeruli with glomerulosclerosis than control rats. This study shows that prenatal dexamethasone in rats results in a reduction in glomerular number, glomerulosclerosis, and hypertension when administered at specific points during gestation. Hypertension was observed in animals that had a reduction in glomeruli as well as in a group that did not have a reduction in glomerular number, suggesting that a reduction in glomerular number is not the sole cause for the development of hypertension.",dexamethasone @CHEMICAL@ hypertension ; increase in blood pressure ; elevated blood pressures @DISEASE@ @CID@ dexamethasone @CHEMICAL@ renal injury ; reduction in glomerular number @DISEASE@ @CID@
1,"Lamivudine for the prevention of hepatitis B virus reactivation in hepatitis-B surface antigen (HBSAG) seropositive cancer patients undergoing cytotoxic chemotherapy. Hepatitis B virus (HBV) is one of the major causes of chronic liver disease worldwide. Cancer patients who are chronic carriers of HBV have a higher hepatic complication rate while receiving cytotoxic chemotherapy (CT) and this has mainly been attributed to HBV reactivation. In this study, cancer patients who have solid and hematological malignancies with chronic HBV infection received the antiviral agent lamivudine prior and during CT compared with historical control group who did not receive lamivudine. The objectives were to assess the efficacy of lamivudine in reducing the incidence of HBV reactivation, and diminishing morbidity and mortality during CT. Two groups were compared in this study. The prophylactic lamivudin group consisted of 37 patients who received prophylactic lamivudine treatment. The historical controls consisted of 50 consecutive patients who underwent CT without prophylactic lamivudine. They were followed up during and for 8 weeks after CT. The outcomes were compared for both groups. Of our control group (n= 50), 21 patients (42%) were established hepatitis. Twelve (24%) of them were evaluated as severe hepatitis. In the prophylactic lamivudine group severe hepatitis were observed only in 1 patient (2.7%) of 37 patients (p < 0.006). Comparison of the mean ALT values revealed significantly higher mean alanine aminotransferase (ALT) values in the control group than the prophylactic lamivudine group; 154:64 (p < 0.32). Our study suggests that prophylactic lamivudine significantly decreases the incidence of HBV reactivation and overall morbidity in cancer patients during and after immunosuppressive therapy. Further studies are needed to determine the most appropriate nucleoside or nucleotide analogue for antiviral prophylaxis during CT and the optimal duration of administration after co

## Load tokenizer and model

In [5]:
model_name = "google/t5-v1_1-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Test tokenizer and model

In [6]:
single_example = dataset_train[0]
single_example

{'input': 'Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests th

In [7]:
tokenizer_output_text = tokenizer(single_example['input'])
print(len(tokenizer_output_text['input_ids']))
tokenizer_output_text

338


{'input_ids': [1823, 24938, 782, 7211, 7, 8, 1181, 13397, 49, 324, 7, 757, 1504, 13, 3, 3903, 29, 30095, 5, 86, 73, 152, 222, 88, 17, 1601, 6, 23496, 120, 6676, 324, 7, 757, 20063, 8, 6313, 16, 1717, 1666, 11, 842, 1080, 2546, 57, 6344, 162, 10529, 3, 3903, 29, 30095, 6, 305, 12, 460, 2179, 5096, 7, 87, 8711, 6, 47, 19921, 15, 26, 42, 7211, 26, 57, 3, 29, 138, 32, 9431, 6, 3, 18189, 12, 204, 5453, 87, 8711, 5, 37, 10950, 324, 7, 757, 1504, 13, 910, 5453, 87, 8711, 491, 6977, 18, 22758, 26, 32, 102, 9, 47, 92, 14610, 7211, 26, 57, 3, 29, 9, 24938, 782, 5, 1823, 24938, 782, 2238, 410, 59, 2603, 893, 1717, 1666, 42, 842, 1080, 5, 86, 2241, 13304, 7, 45, 23496, 120, 6676, 324, 7, 757, 20063, 3, 3903, 29, 30095, 6, 335, 599, 18, 13520, 12, 335, 599, 18, 9120, 283, 6, 410, 59, 2860, 16687, 7, 15, 3437, 757, 11293, 13, 784, 519, 566, 908, 18, 29, 9, 24938, 782, 13642, 3, 29, 329, 201, 11, 3, 29, 9, 24938, 782, 6, 335, 599, 18, 13520, 12, 335, 599, 18, 7256, 283, 6, 410, 59, 2860, 3, 3903, 29,

In [8]:
with tokenizer.as_target_tokenizer():
    tokenizer_output_rel = tokenizer(single_example['relations'])
tokenizer_output_rel

/home/lgrootde/Generative-re-tests/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [491, 6977, 18, 22758, 26, 32, 102, 9, 3320, 13717, 329, 23936, 1741, 10950, 324, 7, 757, 3320, 308, 19056, 17892, 1741, 3320, 254, 4309, 1741, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenized_dataset = tokenizer(dataset_train['input'])
count = 0
for idx, input_ids in enumerate(tokenized_dataset['input_ids']):
    if len(input_ids) > 512:
        # print(idx, len(input_ids))
        count+=1
print(f'Percentage that comes above max sequence length: {count/(idx+1)*100}')

Token indices sequence length is longer than the specified maximum sequence length for this model (619 > 512). Running this sequence through the model will result in indexing errors


Percentage that comes above max sequence length: 13.4


## Create Preprocessing function

In [10]:
ignore_pad_token_for_loss=True

In [11]:
def preprocess_function(examples):
    '''
    This function takes a dataset of input and target sequences.
    meant to be used with the dataset.map() function
    '''

    text_column = 'input'
    rel_column = 'relations'

    # Split input and target
    inputs, targets = [], []
    for i in range(len(examples[text_column])):
        if examples[text_column][i] and examples[rel_column][i]: # remove pairs where one is None
            inputs.append(examples[text_column][i])
            targets.append(examples[rel_column][i])

    # Tokenize the input
    model_inputs = tokenizer(
        inputs, 
        max_length=512, 
        padding=True, 
        truncation=True, 
        return_tensors='pt'
    )

    # Tokenize the target sequence
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, 
            max_length=512, 
            padding=True, 
            truncation=True, 
            return_tensors='pt'
        )

    # Replace pad tokens with -100 so they don't contribute too the loss
    if ignore_pad_token_for_loss:
        labels["input_ids"] = [
                    [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
                ]

    # Add tokenized target text to output
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [12]:
train_dataset = dataset_train.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset"
)

eval_dataset = dataset_eval.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset"
)

Running tokenizer on train dataset: 100%|██████████| 500/500 [00:02<00:00, 223.50 examples/s]


## Create evaluation strategy

In [13]:
import evaluate

In [16]:
metric = evaluate.load("rouge")

We need to define two functions:
- a postprocess function that does the post processing of a prediction
- A compute_metrics function that takes as input an prediction and output the mean score of that prediction?

the compute metrics functions alwas takes a tuple of 

In [20]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

In [21]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    # Replace -100s used for padding as we can't decode them
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result